In [3]:
#struktury ; jedno
#iteracyjne ; trzy

In [4]:
import numpy as np

Rzadkie wektory i macierze:
1. Zaimplementować struktury danych przechowujące rzadkie wektory i macierze (format dowolny)
2. Zaimplementować podstawowe operacje na wektorach (dodawanie, mnożenie przez skalar, przez wektor)

In [219]:
class SparseVector:
    def __init__(self, I: list, V: list, N: int):
        self.I = I
        self.V = V
        self.N = N
    
    def transformToNumpyMatrix(self):
        A = np.zeros(self.N)
        for i in range(0, len(self.I)):
            A[self.I[i]] = self.V[i]
        return A

In [220]:
I1 = [3, 4, 6]
V1 = [3.4, 5.6, 6.7]
N = 10

In [221]:
sparse1 = SparseVector(I1, V1, N)
A = sparse1.transformToNumpyMatrix()
A

array([0. , 0. , 0. , 3.4, 5.6, 0. , 6.7, 0. , 0. , 0. ])

In [222]:
I2 = [1, 4, 7]
V2 = [-5.4, 1.6, 6.7]

In [223]:
sparse2 = SparseVector(I2, V2, N)
B = sparse2.transformToNumpyMatrix()
B

array([ 0. , -5.4,  0. ,  0. ,  1.6,  0. ,  0. ,  6.7,  0. ,  0. ])

In [224]:
def multiplyByScalar(a: SparseVector, b:float):
    for i in range(0, len(a.I)):
        a.V[i] = a.V[i] * b

In [225]:
multiplyByScalar(sparse1, 4)
A = sparse1.transformToNumpyMatrix()
A

array([ 0. ,  0. ,  0. , 13.6, 22.4,  0. , 26.8,  0. ,  0. ,  0. ])

In [228]:
def addSparseVectors(a: SparseVector, b: SparseVector) -> SparseVector:
    C = SparseVector([], [], a.N)
    
    for i in range(0, len(a.I)):
        value = a.V[i]
        for j in range(0, len(a.I)):
            if (a.I[i] == b.I[j]):
                value = value + b.V[j]
        C.I.append(a.I[i])
        C.V.append(value)
    
    for i in range(0, len(a.I)):
        value = b.V[i]
        for j in range(0, len(a.I)):
            if (a.I[i] == b.I[j]):
                value = value + a.V[j]
        if i not in C.I:
            C.I.append(b.I[i])
            C.V.append(value)
    return C

In [229]:
sparseSum = addSparseVectors(sparse1, sparse2)
sparseSum.transformToNumpyMatrix()

array([ 0. , -5.4,  0. , 13.6, 24. ,  0. , 26.8,  6.7,  0. ,  0. ])

Iteracyjne rozwiązywanie układów równań liniowych:
1. Metoda Newtona znajdowania zer funkcji rzeczywistych.

In [12]:
def newton_raphson(f, df, x, nmax, eps, omg):
    fx = f(x)
    for n in range(0, nmax):
        fp = df(x)
        if(abs(fp) < omg):
            print("small derivative")
            return
        d = fx/fp
        x = x - d
        fx = f(x)
        print(n, x, fx)
        if(abs(d) < eps):
            print("convergence")
            return

In [13]:
f = lambda x:  x**5 + 4*x**4 - 2*x**3 - 17
df = lambda x: 5*x**4 + 16*x**3 - 6*x**2

In [14]:
newton_raphson(f, df, 1, 10, 0.0001, 0.0001)

0 1.9333333333333333 51.44175670781894
1 1.6178368465037598 13.017424422434942
2 1.467000689987531 2.006152575260618
3 1.4339824053476466 0.07959835449666741
4 1.4325605853145769 0.0001418648515851828
5 1.4325580422024418 4.530846808847855e-10
convergence


2. Metoda Jacobiego iteracyjnego rozwiązywania układów równań liniowych.

In [15]:
def jacobi(A: np.matrix, b: np.matrix, x: np.matrix):
    maxIter = 100
    omg = 10**(-10)
    n = A.shape[0]

    for k in range(0, maxIter):
        y = x
        for i in range(0, n):
            sum = b[i]
            diag = A[i, i]
            if(abs(diag) < omg):
                print("diagonal element too small")
                return
            for j in range(0, n):
                if i != j:
                    sum = sum -  A[i,j]*y[j]
            x[i] = sum/diag

In [16]:
A = np.matrix([[2.0, -1.0, 0.0],
              [-1.0, 3.0, -1.0],
              [0.0 ,-1.0, 2.0]])
b = np.matrix([[1.0],
               [8.0],
               [-5.0]])
x = np.matrix([[0.0], 
               [0.0], 
               [0.0]])

In [17]:
jacobi(A, b, x)

In [18]:
np.allclose(x, np.linalg.solve(A,b))

True

3. Przetestować powyższą metodę dla wygenerowanych macierzy (2x2, 3x3, 4x4) i sprawdzić jej poprawność.

In [19]:
A2 = np.random.randint(1, 100, (2, 2))
B2 = np.random.randint(1, 100, 2)
X2 = np.zeros(2)

In [20]:
jacobi(A2, B2, X2)
np.allclose(X2, np.linalg.solve(A2, B2))

True

In [21]:
A3 = np.random.randint(1, 100, (3, 3))
B3 = np.random.randint(1, 100, 3)
X3 = np.zeros(3)

In [22]:
jacobi(A3, B3, X3)
np.allclose(X3, np.linalg.solve(A3, B3))

False

In [23]:
A4 = np.random.randint(1, 100, (4, 4))
B4 = np.random.randint(1, 100, 4)
X4 = np.zeros(4)

In [24]:
jacobi(A4, B4, X4)
np.allclose(X4, np.linalg.solve(A4, B4))

False

Metoda Jacobiego jest zbieżna tylko dla macierzy o dominującej przekątnej.